## Etapa 1: Importação das bibliotecas

In [ ]:
from torchvision import datasets, transforms
import torch
from torch import nn, optim
from sklearn.metrics import accuracy_score
torch.__version__

'1.13.1+cu116'

In [ ]:
torch.manual_seed(123)

## Etapa 2: Base de dados

In [ ]:
dataset_train = datasets.MNIST('MNIST-data', train = True, download = True, 
                               transform = transforms.ToTensor())
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 256, 
                                           shuffle = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST-data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST-data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST-data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST-data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST-data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST-data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST-data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST-data/MNIST/raw



In [ ]:
dataset_val = datasets.MNIST('MNIST-data', train = False, download = True, 
                             transform = transforms.ToTensor())
loader_val = torch.utils.data.DataLoader(dataset_val, batch_size = 256, 
                                         shuffle = True)

## Etapa 3: Construção do autoencoder

In [ ]:
# 784 -> 32 -> 784
class autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(in_features = 784, out_features = 32)
        self.activation0 = nn.ReLU()
        self.dense1 = nn.Linear(32, 784)
        self.activation1 = nn.Sigmoid()
    
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dense1(X)
        X = self.activation1(X)
        return X

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
model = autoencoder()
model.to(device)

autoencoder(
  (dense0): Linear(in_features=784, out_features=32, bias=True)
  (activation0): ReLU()
  (dense1): Linear(in_features=32, out_features=784, bias=True)
  (activation1): Sigmoid()
)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

## Etapa 4: Treinamento do modelo

In [ ]:
for epoch in range(20):
    running_loss_train = 0.
    for data in loader_train:
        model.train()
        inputs, _ = data
        inputs = inputs.to(device)
        
        inputs = inputs.view(-1, 28*28)
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, inputs.view(*outputs.shape))
        loss.backward()
       
        optimizer.step()

        running_loss_train += loss.item()
    
    # Validação
    running_loss_val = 0.
    for data in loader_val:
        model.eval()
        inputs, _ = data
        inputs = inputs.to(device)
        inputs = inputs.view(-1, 28*28)
        outputs = model(inputs)
        loss = criterion(outputs, inputs.view(*outputs.shape))
        running_loss_val += loss.item()

    # Final da época
    print('ÉPOCA {:3d}: perda_train {:.5f} perda_val {:.5f}'.format(epoch + 1, running_loss_train/len(loader_train), running_loss_val/len(loader_val)))

ÉPOCA   1: perda_train 0.29529 perda_val 0.19752
ÉPOCA   2: perda_train 0.18119 perda_val 0.16547
ÉPOCA   3: perda_train 0.15759 perda_val 0.14778
ÉPOCA   4: perda_train 0.14333 perda_val 0.13660
ÉPOCA   5: perda_train 0.13427 perda_val 0.12921
ÉPOCA   6: perda_train 0.12780 perda_val 0.12397
ÉPOCA   7: perda_train 0.12312 perda_val 0.11901
ÉPOCA   8: perda_train 0.11805 perda_val 0.11451
ÉPOCA   9: perda_train 0.11506 perda_val 0.11267
ÉPOCA  10: perda_train 0.11310 perda_val 0.11086
ÉPOCA  11: perda_train 0.11184 perda_val 0.10954
ÉPOCA  12: perda_train 0.11099 perda_val 0.10927
ÉPOCA  13: perda_train 0.11038 perda_val 0.10839
ÉPOCA  14: perda_train 0.11003 perda_val 0.10836
ÉPOCA  15: perda_train 0.10971 perda_val 0.10761
ÉPOCA  16: perda_train 0.10814 perda_val 0.10623
ÉPOCA  17: perda_train 0.10751 perda_val 0.10551
ÉPOCA  18: perda_train 0.10714 perda_val 0.10535
ÉPOCA  19: perda_train 0.10689 perda_val 0.10544
ÉPOCA  20: perda_train 0.10675 perda_val 0.10555


## Etapa 5: Geração do encoder

In [ ]:
list(model.children())

[Linear(in_features=784, out_features=32, bias=True),
 ReLU(),
 Linear(in_features=32, out_features=784, bias=True),
 Sigmoid()]

In [ ]:
class encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = list(model.children())[0]
        self.activation0 = list(model.children())[1]
    
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        return X

In [ ]:
model1 = encoder()
model1.to(device)

encoder(
  (dense0): Linear(in_features=784, out_features=32, bias=True)
  (activation0): ReLU()
)

## Etapa 6: Codificação dos previsores

In [ ]:
previsores_treinamento = dataset_train.data.float() / 255
previsores_treinamento = previsores_treinamento.view(-1, 784)
previsores_treinamento = previsores_treinamento.to(device)

In [ ]:
previsores_treinamento.shape

torch.Size([60000, 784])

In [ ]:
with torch.no_grad():
  previsores_treinamento_codificados = model1(previsores_treinamento)
classe_treinamento = dataset_train.targets

In [ ]:
previsores_treinamento_codificados.shape

torch.Size([60000, 32])

In [ ]:
classe_treinamento

tensor([5, 0, 4,  ..., 5, 6, 8])

In [ ]:
dataset_train_cod = torch.utils.data.TensorDataset(previsores_treinamento_codificados, classe_treinamento)
loader_train_cod = torch.utils.data.DataLoader(dataset_train_cod, batch_size = 256, shuffle=True)

In [ ]:
previsores_val = dataset_val.data.float() / 255
previsores_val = previsores_val.view(-1, 784)
previsores_val = previsores_val.to(device)
with torch.no_grad():
  previsores_val_codificados = model1(previsores_val)
classe_val = dataset_val.targets

In [ ]:
previsores_val_codificados.shape

torch.Size([10000, 32])

In [ ]:
dataset_test_cod = torch.utils.data.TensorDataset(previsores_val_codificados, classe_val)
loader_val_cod = torch.utils.data.DataLoader(dataset_test_cod, batch_size = 256, shuffle=True)

## Etapa 7: Classificação sem redução de dimensionalidade

In [ ]:
c1 = nn.Sequential(nn.Linear(784, 397),
                   nn.ReLU(),
                   nn.Linear(397, 10),
                   nn.LogSoftmax())
c1.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(c1.parameters())

In [ ]:
for epoch in range(20):  
    # Treinamento
    running_loss_train = 0.
    running_accuracy_train = 0.
    for data in loader_train:
        c1.train()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 28*28)
        optimizer.zero_grad()
        outputs = c1(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss_train += loss.item()
        running_accuracy_train += accuracy_score(labels.detach().cpu().numpy(), 
                                                 top_class.detach().cpu().numpy())
    
    # Validação
    running_loss_val = 0.
    running_accuracy_val = 0.
    for data in loader_val:
        c1.eval()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 28*28)
        outputs = c1(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        running_loss_val += loss.item()
        running_accuracy_val += accuracy_score(labels.detach().cpu().numpy(), 
                                               top_class.detach().cpu().numpy())
        
    # Final da época
    print('ÉPOCA {:3d}: perda_train {:.5f} precisão_train {:5f} perda_val {:.5f} precisão_val {:5f}'.format(epoch + 1,
            running_loss_train/len(loader_train),
            running_accuracy_train/len(loader_train), 
            running_loss_val/len(loader_val),
            running_accuracy_val/len(loader_val)))

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

ÉPOCA   1: perda_train 0.40669 precisão_train 0.895013 perda_val 0.20284 precisão_val 0.940137


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

ÉPOCA   2: perda_train 0.17420 precisão_train 0.950698 perda_val 0.14295 precisão_val 0.956445


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

...

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

ÉPOCA  19: perda_train 0.00393 precisão_train 0.999701 perda_val 0.06491 precisão_val 0.981348


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

ÉPOCA  20: perda_train 0.00338 precisão_train 0.999834 perda_val 0.06688 precisão_val 0.982031


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

## Etapa 8: Classificação com redução de dimensionalidade

In [ ]:
c2 = nn.Sequential(
        nn.Linear(32, 21),
        nn.ReLU(),
        nn.Linear(21, 10),
        nn.LogSoftmax())
c2.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(c2.parameters())

In [ ]:
for epoch in range(20): 
    running_loss_train = 0.
    running_accuracy_train = 0.
    for data in loader_train_cod:
        c2.train()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = c2(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss_train += loss.item()
        running_accuracy_train += accuracy_score(labels.detach().cpu().numpy(), 
                                                 top_class.detach().cpu().numpy())
    
    # Validação
    running_loss_val = 0.
    running_accuracy_val = 0.
    for data in loader_val_cod:
        c2.eval()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = c2(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        running_loss_val += loss.item()
        running_accuracy_val += accuracy_score(labels.detach().cpu().numpy(), 
                                               top_class.detach().cpu().numpy())
        
    # Final da época
    print('ÉPOCA {:3d}: perda_train {:.5f} precisão_train {:5f} perda_val {:.5f} precisão_val {:5f}'.format(epoch + 1,
            running_loss_train/len(loader_train),
            running_accuracy_train/len(loader_train), 
            running_loss_val/len(loader_val),
            running_accuracy_val/len(loader_val)))

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

ÉPOCA   1: perda_train 1.95911 precisão_train 0.392958 perda_val 1.10482 precisão_val 0.671582


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

ÉPOCA   2: perda_train 0.86181 precisão_train 0.750709 perda_val 0.64985 precisão_val 0.815039


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

...

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

ÉPOCA  19: perda_train 0.27438 precisão_train 0.917825 perda_val 0.25591 precisão_val 0.923047


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch

ÉPOCA  20: perda_train 0.26749 precisão_train 0.920540 perda_val 0.25792 precisão_val 0.920605


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension ch